In [ ]:
import subprocess
import os
import jaccard.jaccard as jac
import binary_ast.compare_ast as bin_ast

cwd = os.getcwd()
os.putenv("GHIDRA_HEADLESS", "/Users/wamuo/Documents/Lab/tools/ghidra_10.1-BETA_PUBLIC/support/analyzeHeadless")

In [ ]:
#Import all packages and at base #20004000 and run ghidra analysis to find functions
#Create a new folder for all analyzed binary using the version number
#For all binaries extract generated instructions for all functions and store in a file named info_graph.json under the version folder
subprocess.call(['./ghidra_mods/ghidra_pipeline.sh', cwd])

In [ ]:
subfolders = [ f.path for f in os.scandir("versions") if f.is_dir() ]
subfolders.reverse()
binary_pairs = [(subfolders[i - 1], subfolders[i]) for i in range(1, len(subfolders))]
print(binary_pairs)

In [ ]:
graph_file_name = "/info_graph.json"

possible_updates_by_binary_pair = {}
for binary_pair in binary_pairs:
    possible_updates = jac.calculate_jaccard(binary_pair[0]+ graph_file_name, binary_pair[1] + graph_file_name, binary_pair[1], 0.85)
    possible_updates_by_binary_pair[binary_pair] = possible_updates
    
    

In [ ]:
for binary_pair in possible_updates_by_binary_pair:
    print(binary_pair)
    binary_a = binary_pair[0].split("/")[1].replace('_', '.')
    binary_b = binary_pair[1].split("/")[1].replace('_', '.')
    func_pairs = possible_updates_by_binary_pair[binary_pair]

    for func_pair in func_pairs:
        print(func_pair)
        func_a = func_pair[1][1]
        func_b = func_pair[1][0]
        
        funcA_ast_file = binary_pair[0] + "/" + func_a + ".json"
        funcB_ast_file = binary_pair[1] + "/" + func_b + ".json"
       
        
        if not os.path.exists(funcA_ast_file):
            subprocess.call(['./ghidra_mods/ast.sh', cwd, binary_a, func_a, binary_pair[0]])
        
        if not os.path.exists(funcB_ast_file):
            subprocess.call(['./ghidra_mods/ast.sh', cwd, binary_b, func_b, binary_pair[1]])
       
        print("\n\n\nNew Comparison \n", func_a, func_b)
        path = binary_pair[1] + "/" + func_a +"_to_" + func_b 
        bin_ast.compare_ast(binary_pair[0], binary_pair[1], funcA_ast_file, funcB_ast_file, path)